# Closure Objects

This post is inspired by Robert Nystorm's excellent book "Crafting Interpreters".
You can find the original idea in here: https://craftinginterpreters.com/closures.html#challenges

## Closures

What is a closure?
Usually, a function defines its own scope. Local variables that are created inside the function - cease to exist when the function returns.

That "cease to exist" means that objects are no longer reachable and they get garbage-collected.
I would like to show you that objects inside functions don't exist after the function returns - but how can I demonstrate that i _cannot_ access an object anymore?

For that we will use the `weakref` python library, which enables us to hold pointers to objects but doesn't prevent them from getting garbage-collected.
When a weakref points to a collected object, it is dramatically announced "dead".

In [1]:
import gc
import weakref

class TrackedObject:     
    pass

def func():
    global ref
    obj = TrackedObject()
    ref = weakref.ref(obj)
    return

func()
gc.collect()
print(ref)  # <weakref at ...; dead>

<weakref at 0x000002A8D3E86750; dead>


See? dead.

However, functions can "close-over" values from outer scopes - making them live even after the function has returned:

In [2]:
def func():
    global ref
    obj = TrackedObject()
    ref = weakref.ref(obj)
    def inner():
        return obj
    return inner

inner_func = func()
gc.collect()
print(ref)  # <weakref at ...; to 'TrackedObject' at ...>

<weakref at 0x000002A8D3E0B790; to 'TrackedObject' at 0x000002A8D3DFA6C0>


Now our tracked object is still alive - that's because it is being used by a _closure_ function, which is still accessable.

## Replacing Objects with Closures

In one of the bonus challenges in his book, Robert Nystorm suggests that objects can actually be replaced by closures. He challenges us to implement a contructor to a Vector object, getters and an addition function - without using `class`, only closures.

This is actually pretty cool:

In [3]:
def vector(x,y):
    def get(i):
        return x if i==0 else y
    return get

def get_x(vec):
    return vec(0)
    
def get_y(vec):
    return vec(1)

def vector_add(vec1, vec2):
    return vector(
        get_x(vec1) + get_x(vec2), get_y(vec1) + get_y(vec2)
    )

def print_vector(vec):
    print(f"({get_x(vec)},{get_y(vec)})")

v1 = vector(1, 2)
v2 = vector(4, 7)
v_sum = vector_add(v1, v2)
print_vector(v_sum)  # (5,9)

(5,9)


What else can we do?

The first thing that came to my mind was python's dunder methods. Can we implement addition of closure-objects with the `+` operator?

With 'normal' python objects, we could to that by implementing an `__add__` method in the class.
So, maybe if we add an `__add__` to our closure-objects we could use `+` on them?

Fortunately, because functions in python are objects themselves, we can freely add attributes and methods to them, so that's what we're gonna try (this is actually a bit cheating, because we take advantage of python `function` objects even though this post is about replacing objects with closures. However, we can't change python built-ins and it _will_ be awesome if we could add objects with `+`)

In [4]:
def addable_vector(x,y):
    def get(i):
        return x if i==0 else y
    get.__add__ = vector_add 
    return get

v1 = addable_vector(1, 2)
v2 = addable_vector(4, 7)
v_sum = v1 + v2
print_vector(v_sum)

TypeError: unsupported operand type(s) for +: 'function' and 'function'

That prints the following error:

_TypeError: unsupported operand type(s) for +: 'function' and 'function'_

Well, that happens because when we do `v1 + v2` what python really does is `type(v1).__add__(v2)` rather than call `__add__` on the `v1` instance itself.
`type(v1)` is a `function`, which doesn't have `__add__` so we get an "unsupported" error.

Why don't we add our `__add__` method to `function` then?

In [5]:
def addable_vector(x,y):
    def get(i):
        return x if i==0 else y
    get.__class__.__add__ = vector_add 
    return get

v1 = addable_vector(1, 2)
v2 = addable_vector(4, 7)
v_sum = v1 + v2
print_vector(v_sum)

TypeError: cannot set '__add__' attribute of immutable type 'function'

Now we get:

_TypeError: cannot set '\_\_add\_\_' attribute of immutable type 'function'_

We have reached a dead end - the designers of python have decided that `function` is an immutable type, and therefore we cannot add dunder methods like normal objects.


## Playing with Time & Space

No doubt that defining objects as closures is less readable than using the traditional `class`.
Closure-objects are also more limited than objects because we can't implement dunder methods to them.
The next question we should ask ourselves is - are they more efficient than objects?


Let's start by measuring the size of closure-objects.
For reference, we implement a class version of our vector:

In [6]:
class Vector:
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def vector_add(self, other):
        return self.__class__(self.x+other.x, self.y+other.y)

    def print_vector(self):
        print(f"({self.x},{self.y})")

Now let's compare the size in bytes using `sys.getsizeof()`:

In [7]:
import sys

closure_vec = vector(1,2)
object_vec=  Vector(1,2)

print(sys.getsizeof(closure_vec))  # 160
print(sys.getsizeof(object_vec))  # 48

160
48


Wow!

The closure-object is more than *3* times bigger than the regular object! Why is that? 

This is the point to clarify something important about `sys.getsizeof()`.
To properly compare the memory consumption of closures and objects we would need a deep dive into how memory is managed inside CPython, the python C compiler - but I feel like it will divert us very far from the main subject of this notebook.
Therefore, we are doing some vanilla memory comparison by using `sys.getsizeof()`.
This function gives us only the size of the object reference (I would say 'pointer', but it feels weird to use 'pointer' when writing about python).
The size of an object we see when using `sys.getsizeof()` is merely the size of the references it holds to internal attributes; these can be viewed by `dir`-ing an object:

In [100]:
dir(object_vec)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'print_vector',
 'vector_add',
 'x',
 'y']

We can see a long list of dunders - these are the object's internal attributes, and in the end the methods and attributes we have defined in the `Vector` class.

To prove my point, we can see that the size of the object's `__dict__` which stores its attributes is already bigger than the "object" size we got when using `sys.getsizeof()` on the object itself:

In [103]:
sys.getsizeof(object_vec.__dict__)

296

This means that the size of an object we see with `sys.getsizeof()` depends on which internal attributes the object contains.

Let's see which attributes can be blamed for extra weight for our closure-object:

In [110]:
object_attrs = dir(object_vec)
closure_attrs = dir(closure_vec)
set(closure_attrs) - set(object_attrs)

{'__annotations__',
 '__builtins__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__',
 '__type_params__'}

Most of these attributes are general, boring references or methods 

% TODO - write about how object attrs are stored in `__dict__` but closure's are nowhere to be found; we re-discover them at `.__closure__[i].cell_contents`

In [120]:
sys.getsizeof(closure_vec.__closure__[1])
# print(closure_vec.__closure__[1].cell_contents)

40

In [9]:
many_kwargs = {'a'+str(i):i for i in range(100000)}

def closure_object(**kwargs):
    def get(i):
        return list(kwargs.values())[i]
    return get

class RegularObject:
    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)

160

In [12]:
def show_size(obj):
    for attr_name in dir(obj):
        if attr_name.startswith("__"):
            attr = getattr(obj, attr_name)
            print(attr_name, sys.getsizeof(attr))

In [78]:
c = closure_object(**many_kwargs)
o = RegularObject(**many_kwargs)

delta = set(dir(c)) - set(dir(o))
print(delta)

show_size(c)
print("***************************")
show_size(o)

{'__annotations__', '__closure__', '__globals__', '__name__', '__qualname__', '__builtins__', '__code__', '__defaults__', '__kwdefaults__', '__type_params__', '__get__', '__call__'}
__annotations__ 64
__builtins__ 6576
__call__ 48
__class__ 432
__closure__ 48
__code__ 256
__defaults__ 16
__delattr__ 48
__dict__ 64
__dir__ 72
__doc__ 16
__eq__ 48
__format__ 72
__ge__ 48
__get__ 48
__getattribute__ 48
__getstate__ 72
__globals__ 6576
__gt__ 48
__hash__ 48
__init__ 48
__init_subclass__ 72
__kwdefaults__ 16
__le__ 48
__lt__ 48
__module__ 49
__name__ 44
__ne__ 48
__new__ 72
__qualname__ 68
__reduce__ 72
__reduce_ex__ 72
__repr__ 48
__setattr__ 48
__sizeof__ 72
__str__ 48
__subclasshook__ 72
__type_params__ 40
***************************
__class__ 1704
__delattr__ 48
__dict__ 3844864
__dir__ 72
__doc__ 16
__eq__ 48
__format__ 72
__ge__ 48
__getattribute__ 48
__getstate__ 72
__gt__ 48
__hash__ 48
__init__ 64
__init_subclass__ 72
__le__ 48
__lt__ 48
__module__ 49
__ne__ 48
__new__ 72
__reduce_

In [92]:
attr_name = "__get__"
attr = getattr(c, attr_name)
print(sys.getsizeof(attr))
print(attr)

48
<method-wrapper '__get__' of function object at 0x000002A8D6EA51C0>


In [25]:
def total_size(obj):
    return sum([sys.getsizeof(getattr(obj,attr)) for attr in dir(obj)])

# print(total_size(closure_vec))
# print(total_size(Vector))
# print(total_size(some_very_long_name(1,2)))
# print(total_size(some_very_long_name))

- No doubt this is less readable than objects
- It is probably weaker, in the sense that we can't add dunder methods (class of closure is always `function`)
- What *is* it good for? need to check memory and runtime of this, compared to regular objects

In [93]:
def func():
    return 

sys.getsizeof(func)

160